### Propuesta indicadores habitabilidad

In [ ]:
rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr,showtext,gt)
remotes::install_github("cpyp-ipdp/funcionespgd", force = TRUE)
library(funcionespgd)

#Fuente para los gráficos
font_add_google("Poppins", "pop")
showtext_auto()

Cargando paquete requerido: pacman

Using GitHub PAT from the git credential store.




rlang  (1.1.4 -> 1.1.6) [CRAN]
glue   (1.7.0 -> 1.8.0) [CRAN]
cli    (3.6.3 -> 3.6.5) [CRAN]
tibble (3.2.1 -> 3.3.0) [CRAN]


Installing 4 packages: rlang, glue, cli, tibble

Warning message:
"package 'tibble' is in use and will not be installed"
Installing packages into 'C:/Users/claud/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'rlang' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'rlang'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problema al copiar C:\Users\claud\AppData\Local\R\win-library\4.4\00LOCK\rlang\libs\x64\rlang.dll  a C:\Users\claud\AppData\Local\R\win-library\4.4\rlang\libs\x64\rlang.dll: Permission denied"
Warning message:
"restored 'rlang'"


...

Warning message:
"cannot remove prior installation of package 'glue'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problema al copiar C:\Users\claud\AppData\Local\R\win-library\4.4\00LOCK\glue\libs\x64\glue.dll  a C:\Users\claud\AppData\Local\R\win-library\4.4\glue\libs\x64\glue.dll: Permission denied"
Warning message:
"restored 'glue'"


package 'cli' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'cli'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problema al copiar C:\Users\claud\AppData\Local\R\win-library\4.4\00LOCK\cli\libs\x64\cli.dll  a C:\Users\claud\AppData\Local\R\win-library\4.4\cli\libs\x64\cli.dll: Permission denied"
Warning message:
"restored 'cli'"



The downloaded binary packages are in
	C:\Users\claud\AppData\Local\Temp\RtmpMZROwB\downloaded_packages
── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file 'C:\Users\claud\AppData\Local\Temp\RtmpMZROwB\remotes762c1ed69e8\cpyp-ipdp-funcionespgd-3fffa52/DESCRIPTION' ... OK
* preparing 'funcionespgd':
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted 'LazyData' from DESCRIPTION
* building 'funcionespgd_0.1.0.tar.gz'



Installing package into 'C:/Users/claud/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



### Pobreza de adecuación sanitaria

In [2]:
# Definir URL base y parte intermedia
url_basica <- "https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte <- "microdatos/enigh"

# Definir los años y los tipos de archivos a descargar
years <- c("2018", "2020", "2022", "2024")
tipos <- c("ns_viviendas", "ns_poblacion")  # Tipos de archivos

# Crear la carpeta de destino si no existe
if (!dir.exists("data")) {
  dir.create("data")
}

# Descargar, descomprimir y renombrar los archivos
for (year in years) {
  for (tipo in tipos) {
    # Construir URL y nombres de archivo
    url <- paste0(url_basica, year, "/", segunda_parte, year, "_", tipo, "_csv.zip")
    zip_destfile <- paste0("data/", year, "_", tipo, ".zip")

    # Descargar solo si el archivo no existe
    if (!file.exists(zip_destfile)) {
      download.file(url, destfile = zip_destfile, mode = "wb")
    } else {
      message("Archivo ya descargado: ", zip_destfile)
    }

    # Extraer archivos en una subcarpeta específica
    extract_dir <- paste0("data/", year, "_", tipo)
    if (!dir.exists(extract_dir)) {
      dir.create(extract_dir)
    }

    unzip(zip_destfile, exdir = extract_dir)

    # Buscar el archivo CSV dentro de la carpeta extraída
    extracted_files <- list.files(extract_dir, pattern = "\\.csv$", full.names = TRUE)

    if (length(extracted_files) == 1) {
      # Determinar el nuevo nombre con formato adecuado
      new_name <- paste0("data/", tipo, "_", year, ".csv")

      # Mover el archivo al directorio principal y renombrarlo
      file.rename(extracted_files, new_name)

      message("Archivo renombrado y movido: ", new_name)
    } else {
      message("Error: No se encontró un archivo CSV único en ", extract_dir)
    }
  }
}


Archivo renombrado y movido: data/ns_viviendas_2018.csv

Archivo renombrado y movido: data/ns_poblacion_2018.csv

Archivo renombrado y movido: data/ns_viviendas_2020.csv

Archivo renombrado y movido: data/ns_poblacion_2020.csv

Archivo renombrado y movido: data/ns_viviendas_2022.csv

Archivo renombrado y movido: data/ns_poblacion_2022.csv

Archivo renombrado y movido: data/ns_viviendas_2024.csv

Archivo renombrado y movido: data/ns_poblacion_2024.csv



In [3]:
estima_pobreza_sanitaria <- function(year, variable) {

#Procesar vivienda
  vivienda <- read_csv(paste0("data/ns_viviendas_", year, ".csv"), show_col_types = FALSE) %>%
  #Abasto de agua
   mutate(
      # --- HOMOLOGACIÓN 2024+ PARA DISP_AGUA ---
      Aj = if (year >= 2024) {
        case_when(
          # Agua entubada dentro -> disp_agua = 1 -> Aj = 3
          agua_ent == 1 ~ 3,
          # Agua entubada fuera (patio/terreno) -> disp_agua = 2 -> Aj = 2
          agua_ent == 2 ~ 2,

          # No entubada (agua_ent == 3): mapear por ab_agua / agua_noe
          (agua_ent == 3) & (ab_agua == 1 | agua_noe == 2) ~ 1,                # llave pública / comunitaria
          (agua_ent == 3) & (ab_agua == 4)                 ~ 1,                # pipa
          (agua_ent == 3) & (ab_agua == 5 | agua_noe == 3) ~ 0,                # de otra vivienda
          (agua_ent == 3) & (ab_agua == 6 | agua_noe == 5) ~ 0,                # de la lluvia
          (agua_ent == 3) & (ab_agua %in% c(2,3,7) | agua_noe %in% c(1,4,6)) ~ 0,  # pozo / río-lago / otro

          TRUE ~ NA_real_
        )
      } else {
        case_when(
          disp_agua %in% c(7, 5, 4) ~ 0,
          disp_agua %in% c(3, 6)    ~ 1,
          disp_agua == 2            ~ 2,
          disp_agua == 1            ~ 3,
          TRUE ~ NA_real_
        )
      },
      AAj = Aj / 3,

###Frecuencia de agua
Frec_Aguaj = case_when(
        dotac_agua == 1 ~ 4.0,
        dotac_agua == 2 ~ 2.0,
        dotac_agua == 3 ~ 1.2,
        dotac_agua == 4 ~ 0.6,
        dotac_agua == 5 ~ 0.4,
        TRUE ~ NA_real_
      ),
AFAj = Frec_Aguaj / 4,

###índice de adecuación y frecuencia de agua
      AAHj = case_when(
        Aj %in% c(0, 1) ~ AAj,
        Aj %in% c(2, 3) ~ AAj * AFAj,
        TRUE ~ NA_real_
      ),

####Índice de adecuación de drenaje
      Drj  = case_when(
        drenaje %in% c(1, 2) ~ 1,
        drenaje %in% c(3, 4, 5) ~ 0,
        TRUE ~ NA_real_
      ),
      ADrj = Drj/1,

###Índice de adecuación de excusado
      EXj = if (year >= 2024) {
        # 2024+: 1=Taza de baño, 2=Letrina, 3=No tienen
        case_when(
          as.numeric(excusado) %in% c(2, 3) ~ 0,                                      # letrina o no tienen -> no cumple
          as.numeric(excusado) == 1 & sanit_agua == "3" ~ 0,                          # taza pero sin descarga
          as.numeric(excusado) == 1 & uso_compar == "1" & sanit_agua == "2" ~ 2,      # compartido + sin descarga directa
          as.numeric(excusado) == 1 & uso_compar == "1" & sanit_agua == "1" ~ 3,      # compartido + descarga directa
          as.numeric(excusado) == 1 & uso_compar == "2" & sanit_agua == "2" ~ 3,      # exclusivo + sin descarga directa
          as.numeric(excusado) == 1 & uso_compar == "2" & sanit_agua == "1" ~ 4,      # exclusivo + descarga directa (norma)
          TRUE ~ NA_real_
        )
      } else {
        # Hasta 2022: 1=Sí, 2=No
        case_when(
          excusado == "2" ~ 0,
          sanit_agua == "3" ~ 0,
          uso_compar == "1" & sanit_agua == "2" ~ 2,
          uso_compar == "1" & sanit_agua == "1" ~ 3,
          uso_compar == "2" & sanit_agua == "2" ~ 3,
          uso_compar == "2" & sanit_agua == "1" ~ 4,
          TRUE ~ NA_real_
        )
      },
      AEXj = EXj / 4) %>%

   #Seleccionar las variables requeridas
    {
      if(year < 2022) select(., folioviv,Aj, AAj, Frec_Aguaj, AFAj, AAHj, Drj, ADrj, EXj, AEXj,upm, est_dis, factor) else
        select(., folioviv, Aj, AAj, Frec_Aguaj, AFAj, AAHj, Drj, ADrj, EXj, AEXj)
    }

 pob <- read_csv(paste0("data/ns_poblacion_", year, ".csv"), show_col_types = FALSE) %>%
    janitor::clean_names() %>%
    # Filtrar parentescos que no pertenecen al nucleo familiar
    filter(!(parentesco >= 400 & parentesco < 500) & !(parentesco >= 700 & parentesco < 800)) %>%
    #Crear clave de entidad
    mutate(
      cve_ent = case_when(
        nchar(folioviv) == 9 ~ paste0("0", substr(folioviv, 1, 1)),
        nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
      )
    ) %>%
    #Pegar vivienda
    left_join(vivienda, by = "folioviv") %>%
    mutate(
      ASj = AAHj * ADrj * AEXj,     # Adecuación sanitaria consolidada
      CSj = 1 - ASj,                # Carencia sanitaria
      year = year,
###Crear estratos de Adecuación sanitaria
E_CSj=case_when(
    CSj<=-0.5 ~ 6,
    CSj>-0.5 & CSj<=-0.1 ~ 5,
    CSj>-0.1 & CSj<=0 ~ 4,
    CSj>0 & CSj<=(1/3) ~ 3,
    CSj>(1/3) & CSj<=0.5 ~ 2,
    CSj>0.5 & CSj<=1 ~ 1,
    TRUE ~ NA_real_),

    ###Crear pobreza de adecuación sanitaria
pobreza_csj = case_when(
  E_CSj >= 4 ~ 0,
  E_CSj <= 3 ~ 1,
  TRUE ~ NA_real_
))

  
#Diseño muestral
  diseno <- pob %>%
    as_survey_design(
      ids = upm,
      strata = est_dis,
      weights = factor,
      nest = TRUE
    )



#Estimaciones de acuerdo a variable solicitada
resultado<-if (variable == "pobreza_sanitaria"){
  diseno %>%
    filter(cve_ent == "09") %>%
    group_by(pobreza_csj) %>%
    summarise(
      pob = survey_total(vartype = "cv"),
      porcentaje = survey_prop(vartype = "cv")
    ) %>%
    mutate(porcentaje = porcentaje * 100, year = year) %>%
    filter(pobreza_csj == 1)
}
else {stop("Variable no reconocida")

}
return(resultado
  )
}

In [4]:
tabla_pobreza_as <- map_dfr(years, estima_pobreza_sanitaria, variable = "pobreza_sanitaria")
tabla_pobreza_as

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or mo

pobreza_csj,pob,pob_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,3725292,0.05549573,40.50876,0.04689366,2018
1,3226613,0.05296163,34.91762,0.04830021,2020
1,3254901,0.05759466,34.98197,0.05351024,2022
1,3467413,0.05502042,37.10223,0.04919279,2024


##### Satisfacción con parques y jardines

In [5]:
url_encig_basica<-"https://www.inegi.org.mx/contenidos/programas/encig/"
segunda_parte<-"microdatos/encig"

In [6]:

years_encig<-c("2017","2019","2021", "2023")

In [7]:
# Descargar los archivos, descomprimirlos y renombrarlos
#timeout de 300

options(timeout = 600)
for (i in 1:length(years_encig)) {
  # Ajustar el sufijo de la URL y archivo según el año
  if (years_encig[i] %in% c("2017", "2019")) {
    fin <- "_base_datos_dbf.zip"
  } else {
    fin <- "_base_datos_csv.zip"
  }

  # Construir la URL y el nombre del archivo destino
  url <- paste0(url_encig_basica, years_encig[i], "/", segunda_parte, str_sub(years_encig[i], 3, 4), fin)
  print(url)
  destfile <- paste0("data/encig", str_sub(years_encig[i], 3, 4), fin)
  
  # Descargar el archivo
  download.file(url, destfile = destfile)
  
  # Descomprimir el archivo
  unzip(destfile, exdir = "data")
}

[1] "https://www.inegi.org.mx/contenidos/programas/encig/2017/microdatos/encig17_base_datos_dbf.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/encig/2019/microdatos/encig19_base_datos_dbf.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/encig/2021/microdatos/encig21_base_datos_csv.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/encig/2023/microdatos/encig23_base_datos_csv.zip"


In [10]:
# Funcion de estimación
process_files <- function(years, variable) {
  # Función anidada para estimar el indicador`
  estima_indicador <- function(path, variable) {
    # Extraer el año del archivo
    year <- substr(path, 11, 14)

    # Leer los datos y hacer las transformaciones necesarias de las variables
    if (year %in% c("2017", "2019")) {
      datos <- foreign::read.dbf(path) %>%
        janitor::clean_names() %>%
mutate(p4_4a= as.numeric(p4_4a),
      fac_p18 = as.numeric(fac_p18),
        upm = as.numeric(upm),
        est_dis = as.numeric(est_dis)) %>%
#Filtrar Ciudad de México
        filter(ent == "09")
    } else if (year %in% c("2021", "2023")) {
      datos <- read_csv(path,show_col_types=FALSE) %>%
        janitor::clean_names() %>%
#Filtrar Ciudad de México
        filter(nom_ent == "CIUDAD DE MEXICO")
    } else {
      stop("Año no soportado.")
    }

    # Diseño muestral
    dm <- datos %>%
      as_survey_design(ids = upm, strata = est_dis, weights = fac_p18)

    # Calidad del agua
    if (variable == "satisf_parques") {
      satisf_pyj <- dm %>%
mutate(satisf=case_when((p4_4a==1 | p4_4a==2) ~ "Satisfecho",
    TRUE ~ "No satisfecho"))%>%
        group_by(satisf) %>%
        summarise(
          personas = survey_total(vartype = "cv"),
          porcentaje = survey_prop(vartype = "cv")
        ) %>%
ungroup()%>%
        mutate(porcentaje = round(porcentaje * 100, 2),
              year = year)%>%
filter(satisf=="Satisfecho")
      return(satisf_pyj)

    }  else {
      stop("Variable no reconocida.")
    }
  }

  map_dfr(years, function(year) {
    # File path de los archivos
    file_path <- if (year %in% c(2017, 2019)) {
      paste0("data/encig", year, "_01_sec1_3_4_5_8_9_10.dbf")
    } else if (year %in% c(2021, 2023)) {
      paste0("data/encig", year, "_01_sec1_A_3_4_5_8_9_10.csv")
    } else {
      stop("Año no soportado.")
    }

    # Print del archivo que se está procesando
    print(paste("Procesando archivo:", file_path))

    # Invocar la función de estimación
    estima_indicador(file_path, variable)
  })
}

In [11]:

tabla_confianza<-process_files(c(2017,2019,2021,2023), variable = "satisf_parques")
print("Porcentaje de personas de 18 años y más que están satisfechas con parques y jardines")
tabla_confianza

[1] "Procesando archivo: data/encig2017_01_sec1_3_4_5_8_9_10.dbf"
[1] "Procesando archivo: data/encig2019_01_sec1_3_4_5_8_9_10.dbf"
[1] "Procesando archivo: data/encig2021_01_sec1_A_3_4_5_8_9_10.csv"


New names:
• `` -> `...1`


[1] "Procesando archivo: data/encig2023_01_sec1_A_3_4_5_8_9_10.csv"


New names:
• `` -> `...1`


[1] "Porcentaje de personas de 18 años y más que están satisfechas con parques y jardines"


satisf,personas,personas_cv,porcentaje,porcentaje_cv,year
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Satisfecho,2234247,0.03719761,25.48,0.03420882,2017
Satisfecho,2282410,0.02793191,32.89,0.02568254,2019
Satisfecho,2630879,0.02514460,37.81,0.02334756,2021
Satisfecho,3389676,0.02415645,44.74,0.02025601,2023
